In [4]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import imblearn
import collections

import sklearn
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.neural_network import MLPClassifier

from sklearn.metrics import classification_report,confusion_matrix

from keras.utils import to_categorical

from keras.utils.vis_utils import plot_model

from collections import Counter
from imblearn.under_sampling import OneSidedSelection
from imblearn.over_sampling import ADASYN
import pickle

Load the Data

In [5]:
csv_file = 'TBharatUMomo_alldata_final_doaction.csv'
dataframe = pd.read_csv(csv_file)

In [6]:
dataframe.head()

print(dataframe["Time Since Last Charge"].mean())
print(dataframe["Time Since Last Action Formatted"].mean())

15.859564089551112
1.746871792702913


Format and One-hot encode the data

In [7]:
dataframe.drop("tsla1", axis=1, inplace=True)
dataframe.drop("tsla2", axis=1, inplace=True)
dataframe.drop("tsla3", axis=1, inplace=True)
dataframe.drop("tsla4", axis=1, inplace=True)
dataframe.drop("tsla5", axis=1, inplace=True)
dataframe.drop("tsla6", axis=1, inplace=True)
dataframe.drop("st1", axis=1, inplace=True)
dataframe.drop("st2", axis=1, inplace=True)
dataframe.drop("st3", axis=1, inplace=True)
dataframe.drop("st4", axis=1, inplace=True)
dataframe.drop("st5", axis=1, inplace=True)
dataframe.drop("st6", axis=1, inplace=True)
dataframe.drop("st7", axis=1, inplace=True)
dataframe.drop("st8", axis=1, inplace=True)
dataframe.drop("st9", axis=1, inplace=True)
dataframe.drop("st10", axis=1, inplace=True)
dataframe.drop("Session Time", axis=1, inplace=True)
dataframe.drop("Time Since Last Action", axis=1, inplace=True)
dataframe.drop("Action", axis=1, inplace=True)

dataframe.drop("Session Time Formatted", axis=1, inplace=True)
#dataframe.drop("Time Since Last Charge", axis=1, inplace=True)  

In [8]:
dataframe.head(100)

,Do Action,Last Action,Time Since Last Charge,Time Since Last Action Formatted,Camera Mood,Gaze,Noise,Input Mood,Activity
0,1,17,0.00,0.02,Sad,1,None,Neutral,Relaxing
1,0,7,0.00,0.00,Sad,1,Low,Neutral,Relaxing
2,0,7,0.00,0.00,Sad,1,None,Neutral,Relaxing
3,0,7,0.00,0.00,Sad,1,None,Neutral,Relaxing
4,0,7,0.01,0.00,Sad,1,None,Neutral,Relaxing
...,...,...,...,...,...,...,...,...,...
95,0,7,0.31,0.30,Neutral,0,Low,Neutral,Relaxing
96,0,7,0.31,0.32,Neutral,0,Low,Neutral,Relaxing
97,0,7,0.31,0.32,Neutral,0,Low,Neutral,Relaxing
98,0,7,0.31,0.32,Neutral,0,Low,Neutral,Relaxing


In [9]:
dataframe = pd.get_dummies(dataframe, columns=["Activity"], prefix=["Activity"])
dataframe = pd.get_dummies(dataframe, columns=["Camera Mood"], prefix=["Camera_Mood"])
dataframe = pd.get_dummies(dataframe, columns=["Input Mood"], prefix=["Input Mood"])
#both = pd.concat([dataframe, dum_df])
#dataframe.drop("Input Mood", axis=1, inplace=True) 
#both.head(5)

dataframe = pd.get_dummies(dataframe, columns=["Noise"], prefix=["Noise"])
dataframe = pd.get_dummies(dataframe, columns=["Last Action"], prefix=["Last Action"])
dataframe.head(100)

,Do Action,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,...,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
0,1,0.00,0.02,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,0,0.00,0.00,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0.00,0.00,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0.00,0.00,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0.01,0.00,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0.31,0.30,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0.31,0.32,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0.31,0.32,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0.31,0.32,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
X = dataframe.drop('Do Action', axis=1)
y = dataframe['Do Action']

In [11]:
#X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)
X_train = X[0:52992]
X_test = X[52992:70932]
y_train = y[0:52992]
y_test = y[52992:70932]

In [12]:
X_test

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Camera_Mood_Surprise,...,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
52992,0.00,0.02,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
52993,0.00,0.00,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
52994,0.00,0.00,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
52995,0.01,0.00,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
52996,0.01,0.00,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70927,12.73,0.75,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
70928,12.73,0.75,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
70929,12.73,0.75,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
70930,12.73,0.75,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
counter = Counter(y_train)
print(counter)

Counter({0: 52817, 1: 175})


In [14]:
counter = Counter(y_test)
print(counter)

Counter({0: 17877, 1: 63})


In [15]:
X_train_scaled = X_train.copy()

col_names = ['Time Since Last Charge', 'Time Since Last Action Formatted']
features = X_train_scaled[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
X_train_scaled[col_names] = features

filename = 'TBharatUMomo_scaler1.sav'
#pickle.dump(scaler, open(filename, 'wb'))

In [16]:
pd.set_option('display.max_columns', None)
X_train_scaled.head()

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Camera_Mood_Surprise,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
0,-1.410104,-0.639670,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,-1.410104,-0.646987,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,-1.410104,-0.646987,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,-1.410104,-0.646987,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,-1.409134,-0.646987,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [17]:
scaler2 = pickle.load(open(filename, 'rb'))

X_test_scaled = X_test.copy()

features = X_test_scaled[col_names]
features = scaler2.transform(features.values)
X_test_scaled[col_names] = features

X_test_scaled.head()

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Camera_Mood_Surprise,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
52992,-1.410104,-0.639670,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
52993,-1.410104,-0.646987,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52994,-1.410104,-0.646987,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52995,-1.409134,-0.646987,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52996,-1.409134,-0.646987,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


OSS Undersampling

In [18]:
# define the undersampling method
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=200)
# transform the dataset
X_train_scaled, y_train = undersample.fit_resample(X_train_scaled, y_train)
# summarize the new class distribution
counter = Counter(y_train)
print(counter)

Counter({0: 18617, 1: 175})


SMOTE ADASYN Oversampling

In [19]:
from imblearn.over_sampling import SMOTE

# define the undersampling method
oversample = SMOTE()        ####################33
# transform the dataset
X_train_scaled, y_train = oversample.fit_resample(X_train_scaled, y_train)
# summarize the new class distribution
counter = Counter(y_train)
print(counter)

Counter({0: 18617, 1: 18617})


Load and Train the Model

In [20]:
mlp = MLPClassifier(max_iter=10000, activation='relu', hidden_layer_sizes=(15)) #doesn't converge at 500 so put to higher, do we want it to converge?
mlp

MLPClassifier(hidden_layer_sizes=15, max_iter=10000)

In [21]:
mlp.fit(X_train_scaled,y_train)

MLPClassifier(hidden_layer_sizes=15, max_iter=10000)

In [102]:
#filename = 'TBharatUMomo_model1.sav'
#pickle.dump(mlp, open(filename, 'wb'))

In [22]:
pred = mlp.predict(X_test_scaled)
pred

array([1, 0, 0, ..., 0, 1, 0])

In [23]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94     17877
           1       0.01      0.33      0.02        63

    accuracy                           0.89     17940
   macro avg       0.50      0.61      0.48     17940
weighted avg       0.99      0.89      0.94     17940



In [24]:
y_test.value_counts()

0    17877
1       63
Name: Do Action, dtype: int64

In [25]:
from collections import Counter
Counter(pred)

Counter({1: 1940, 0: 16000})

In [26]:
prediction = pd.DataFrame(pred, columns=['predictions']).to_csv('predictmlp.csv')